<a href="https://colab.research.google.com/github/kartik2627/GENAI/blob/main/rag_chatbot_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 RAG Chatbot on Google Colab
This notebook builds a Retrieval-Augmented Generation chatbot using:
- LangChain
- OpenAI GPT (GPT-3.5 or GPT-4)
- FAISS for vector search
- PDF/Text as knowledge base


In [ ]:
!pip install langchain openai faiss-cpu tiktoken PyPDF2 python-dotenv

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Example PDF - Replace with your own file
!wget -O example.pdf https://arxiv.org/pdf/2304.03442.pdf

loader = PyPDFLoader("example.pdf")
pages = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(pages)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embedding = OpenAIEmbeddings()
vectordb = FAISS.from_documents(docs, embedding)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    return_source_documents=True
)

In [ ]:
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    result = qa_chain(query)
    print("Bot:", result['result'])